In [ ]:
import re 
import json
def parse(answer:str,args):
    pattern = r"<text>(.*?)</text>\s*<target>(.*?)</target>\s*<ground_truth>(.*?)</ground_truth>"
    matches = re.findall(pattern, answer)
    ans = []
    for match in matches:
        ans.append({
            "text":match[0],
            "target":match[1],
            "ground_truth":match[2]
        })
    if len(ans) != args.spurious_num*args.generate_num:
        return []
    return ans
class Args:
    def __init__(self):
        self.spurious_num = 3
        self.generate_num = 10
path = "/home/ubuntu/wcc/now-task/data/train/prompt2_3.1_7_3_0.8/iter3_raw_response.json"
with open(path,"r") as f:
    data = json.load(f)
args = Args()
parse_data = []
for i in data:
    parse_data += parse(i,args)
print(parse_data)

In [ ]:
from response import get_batch_response
import json
import re
model = "meta.llama3-1-8b-instruct-v1:0"

def format_prompts(data):
    # prompt = """
    # I am training a model using RoBERTa + MLP on a task named {task_name}. The task involves {task_description}. 
    # Your task is to identify potential spurious patterns that the model might have learned based on its responses.

    # I will present you with an instance where the model provided incorrect responses. 

    # Please provide {spurious_num} assumptions of spurious patterns that may have caused the incorrect response. And you should provide
    # how to generate corresponding training data about every spurious pattern in those patterns.
    
    # A spurious pattern refers to a misleading or non-causal feature relationship that the model learns during training, 
    # such as misunderstandings of certain phrases, sentiment words, or entity relations. Be specific in the patterns, such as what words or what relations, but not a general description.

    # Format your evaluation instances using XML tags <Spurious_i> and </Spurious_i>. 

    # The incorrect instance is as follows:
    # {data}
    # Please output all content completely without omitting or summarizing.

    # """
    prompt = """
    I am training a model using RoBERTa + MLP on a task named {task_name}. The task involves {task_description}. 
    Your task is to identify potential spurious patterns that the model might have learned based on its responses.

    I will present you with an instance where the model provided incorrect responses.

    Please provide {spurious_num} assumptions of spurious patterns that may have caused the incorrect response. For each spurious pattern, also provide a detailed strategy for generating corresponding training data to test or mitigate the identified spurious pattern.
    
    The “generate strategy” should induce the model to increase the diversity of the generated data as much as possible.

    A spurious pattern refers to a misleading or non-causal feature relationship that the model learns during training, such as misunderstandings of certain phrases, sentiment words, or entity relations. Be specific in the patterns, such as what words or what relations, and avoid general descriptions.

    The incorrect instance is as follows:
    {data}
    Please ensure the output is formatted as follows:

    ```xml
    <SpuriousPatterns>
        <Spurious_1>
            <Pattern>Description of spurious pattern 1 (specific and detailed)</Pattern>
            <GenerateStrategy>Detailed strategy to generate training data for spurious pattern 1</GenerateStrategy>
        </Spurious_1>
        <Spurious_2>
            <Pattern>Description of spurious pattern 2 (specific and detailed)</Pattern>
            <GenerateStrategy>Detailed strategy to generate training data for spurious pattern 2</GenerateStrategy>
        </Spurious_2>
        ...
    </SpuriousPatterns>

    """
   
    task_name = "stance detection"
    task_description = "Stance detection aims to identify the authors' attitudes or positions [FAVOR, NONE, AGAINST] towards a specific target such as an entity, a topic."
    spurious_num = 3
    # generate_num = 
    return prompt.format(task_name=task_name,task_description=task_description,data=data,spurious_num=spurious_num)
def parse(s:str):
    pattern_regex = re.compile(r"<Pattern>(.*?)</Pattern>\s*<GenerateStrategy>(.*?)</GenerateStrategy>",re.DOTALL)

    # 匹配内容
    matches = pattern_regex.findall(s)
    ans = []
    # 提取结果
    for i, (pattern, generate_strategy) in enumerate(matches, 1):
        # print(f"Spurious_{i}:")
        # print(f"Pattern: {pattern.strip()}")
        # print(f"GenerateStrategy: {generate_strategy.strip()}")
        # print("-" * 50)
        ans.append({
            "Pattern":pattern.strip(),
            "GenerateStrategy": generate_strategy.strip()
        })
    return ans
def parse2(s:str):
    pattern = r"<text>(.*?)</text>\s*<target>(.*?)</target>\s*<ground_truth>(.*?)</ground_truth>"
    matches = re.findall(pattern, s)
    ans = []
    for match in matches:
        ans.append({
            "text":match[0],
            "target":match[1],
            "ground_truth":match[2]
        })
    # if len(ans) != args.spurious_num*args.generate_num:
    #     return []
    return ans
def format_output_prompt(spurious,generate_strategy):
    prompt = """
    
    I am training a model using RoBERTa + MLP on a task named {task_name}. The task involves {task_description}. 
    Your task is to generate diverse and contextually appropriate training data based on the provided spurious pattern and generation strategy.

    The spurious pattern and corresponding generation strategy are as follows:
    <Spurious>
        <Pattern>{spurious}</Pattern>
        <GenerateStrategy>{generate_strategy}</GenerateStrategy>
    </Spurious>

    Based on the provided spurious pattern and generation strategy, generate {generate_num} verification data points. 
    Each generated data point should align with the spurious pattern and adhere to the specified generation strategy.

    ### Output Format:
    Each verification data point should be structured as follows:
    - <verification_i> for each data point, where `i` is the sequential number of the verification set.

    Each <verification_i> should contain the following:
    1. <text>: A multi-sentence passage (at least 100 words) containing the spurious pattern within a suitable context. 
    The passage should demonstrate the identified spurious pattern while maintaining coherence and diversity.
    2. <target>: An entity or phrase from the text that is the focus of the classification task.
    3. <ground_truth>: The true label for the classification task, ensuring logical consistency with the provided text, you can just use one of ["FAVOR","AGAINST","NONE"].

    Ensure that the generated data points are diverse, use various speaking styles, and include different entities and contexts to avoid overfitting during model fine-tuning.

    ### Example Output:
    <verification_1>
        <text>"She always goes the extra mile to assist her colleagues and solve problems effectively."</text>
        <target>"colleagues"</target>
        <ground_truth>FAVOR</ground_truth>
    </verification_1>
    <verification_2>
        <text>"The weather always changes unpredictably in this region, making planning difficult."</text>
        <target>"weather"</target>
        <ground_truth>NONE</ground_truth>
    </verification_2>
    <verification_3>
        <text>"He always delays submitting his reports, which causes unnecessary delays in the project."</text>
        <target>"reports"</target>
        <ground_truth>AGAINST</ground_truth>
    </verification_3>

    ### Now, generate the output based on the following inputs:
    <Spurious>
        <Pattern>{spurious}</Pattern>
        <GenerateStrategy>{generate_strategy}</GenerateStrategy>
    </Spurious>

    """
    task_name = "stance detection"
    task_description = "Stance detection aims to identify the authors' attitudes or positions [FAVOR, NONE, AGAINST] towards a specific target such as an entity, a topic."
    generate_num = 10
    return prompt.format(task_name=task_name,task_description=task_description,generate_num=generate_num,spurious=spurious,generate_strategy=generate_strategy)

with open("./results/baseline/checkpoint-318/log_dev_wrong_test_data.json","r") as f:
    data = json.load(f)
prompts = []
for i in range(10):
    prompts.append(format_prompts(data=data[i]))            
# print(get_batch_response(model,[prompt])[0])
print(len(prompts))
answer = get_batch_response(model,prompts)

patterns = []
for i in answer:
    # print(i)
    # print("--------")
    patterns.extend(parse(i))

prompt2 = []
for i in range(10):
    prompt2.append(format_output_prompt(patterns[i]["Pattern"],patterns[i]["GenerateStrategy"]))
    
answer2 = get_batch_response(model,prompt2)
parse_output = []
for i in answer2:
    # print(i)
    parse_output.extend(parse2(i))
print(parse_output)

In [ ]:
import os
def get_model_path(path:str):
    paths = []
    for root, dirs, files in os.walk(path):
        # print(dirs)
        dirs.sort()
        for dir_name in dirs:
            
            if dir_name == "logs":
                continue
            # print(dirs)
            dir_path = os.path.join(root, dir_name)
            # print("Folder:", dir_path)            
            paths.append(dir_path)
    return paths
get_model_path('../results/strategy1_3.1_7_3_0.0_iter1/')

In [64]:
import pandas as pd
import csv
combined_train_path = "/home/ubuntu/wcc/now-task/data/train/strategy1_3.1_7_3_0.0_generate5_augmetation1_rate3/iter1_raw_response_parse_combined.csv"
with open(combined_train_path, mode='r', encoding='utf-8') as file:
    reader = csv.DictReader(file) 
    data = [row for row in reader]
# print(data)
for i in data:
    del i["seen?"]
    del i["GT Target"]
    i["text"] = i["Tweet"]
    del i["Tweet"]
data = data[-10:]
for i in data:
    print(i," ")

{'Target 1': 'kurdish people', 'Stance 1': 'FAVOR', 'text': "The international community has a moral obligation to protect the Kurdish people from genocide and persecution. The Turkish government's actions against the Kurds are a clear violation of human rights and international law, and they must be held accountable."}  
{'Target 1': 'syrian civil war', 'Stance 1': 'NONE', 'text': "The Syrian civil war has been a devastating conflict that has caused immense human suffering and destruction. It's time for all parties involved to put aside their differences and work towards a peaceful resolution that respects the rights and dignity of all Syrians."}  
{'Target 1': 'company', 'Stance 1': 'AGAINST', 'text': 'Regulation of corporations has been subverted by corporations. States that incorporate corporations are not equipped to regulate corporations that are rich enough to influence elections, are rich enough to muster a legal team that can bankrupt the state. Money from corporations and the

In [ ]:
from response import get_batch_response
import json
import re
model = "meta.llama3-1-8b-instruct-v1:0"
def format_prompt(k,data):
    prompt = """
    You are given the following JSON data:

    {data}

    Your task is to generate {k} new XML examples that follow the same structure as the provided JSON. Each new example must have the same elements as the original JSON data . The new examples should reflect the underlying patterns of the original data while ensuring diversity. 

    For each new example:
    1. The structure of the XML should match the original JSON, with the same elements.
    2. The values should be different, but the format and types (e.g., numbers, strings, lists) should remain consistent with the original data.
    3. The new XML data should be valid and realistic for the context of the task.
    4. The ground_truth should be one of [FAVOR,NONE,AGAINST].

    Please output the generated data in the following XML format, where each element corresponds to a key in the original JSON:

    ```xml
    <data>
        <example_1>
            <text>Regulation of corporations has been subverted by corporations. States that incorporate corporations are not equipped to regulate corporations that are rich enough to influence elections, are rich enough to muster a legal team that can bankrupt the state. Money from corporations and their principals cannot be permitted in the political process if democracy is to survive.</text>
            <target>company</target>
            <ground_truth>AGAINST</ground_truth>
        </example_1>
        <example_2>
            <text>The whole media mess surrounding the royals is a consequence of the promotional fervor with which royal households (aka, public relations experts) developed stage-set performances for the public to devour. Prior to the Victorian era, those elaborate and lethally expensive weddings, coronations, and funerals - and the fairy tales that went along with them - just didn't exist.</text>
            <target>flag burning</target>
            <ground_truth>NONE</ground_truth>
        </example_2>
        <example_3>
            <text>Two of the main reasons people switch to cannabis from pharmaceuticals and other drugs such as alcohol: less side-effects and less withdrawal: ""Over 41% state that they use cannabis as a substitute for alcohol, 36.1% use cannabis as a substitute for illicit substances, and 67.8% use cannabis as a substitute for prescription drugs. The three main reasons cited for cannabis-related substitution are 'less withdrawal' (67.7%), 'fewer side-effects' (60.4%), and 'better symptom management' suggesting that many patients may have already identified cannabis as an effective and potentially safer adjunct or alternative to their prescription drug regimen."" [Lucas et al. Cannabis as a substitute for alcohol and other drugs: A dispensary-based survey of substitution effect in Canadian medical cannabis patients. Addiction Research & Theory. 2013]</text>
            <target>marijuana</target>
            <ground_truth>FAVOR</ground_truth>
        </example_3>
        ...
    </data>

    """
    return prompt.format(k=k,data=data)
path = "/home/ubuntu/wcc/now-task/data/train/strategy1_3.1_7_3_0.0_generate1/iter1_raw_response_parse.json"
with open(path,"r") as f:
    data = json.load(f)
# for d in data:
#     del d["target"]
k = 5
print(data[0])
prompts = []
for i in data:
    prompts.append(format_prompt(k,i))
prompts = prompts[:10]
response = get_batch_response(model=model,prompts=prompts)
# for i in response:
#     print(i)
def parse2(s:str):
    pattern = r"<text>(.*?)</text>\s*<target>(.*?)</target>\s*<ground_truth>(.*?)</ground_truth>"
    matches = re.findall(pattern, s)
    ans = []
    for match in matches:
        ans.append({
            "text":match[0],
            "target":match[1],
            "ground_truth":match[2]
        })
    return ans
parse_data = []
for i in response:
    parse_data.extend(parse2(i))
print(len(parse_data))
for i in parse_data:
    print(i)